In [2]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [3]:
# Set plot style
sns.set(style='whitegrid')


In [6]:
df = pd.read_csv("../data/raw/MachineLearningRating_v3.txt",
                 error_bad_lines=False, 
                 warn_bad_lines=True)


TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

In [ ]:
# --- Data Overview ---
print("\n--- Dataset Info ---")
print(df.info())
print("\n--- Descriptive Statistics ---")
print(df.describe(include='all'))

In [4]:
# --- Data Quality Checks ---
print("\n--- Missing Values ---")
missing = df.isnull().sum().sort_values(ascending=False)
print(missing[missing > 0])


--- Missing Values ---


NameError: name 'df' is not defined

In [5]:
# --- Feature Engineering ---
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']


NameError: name 'df' is not defined

In [6]:

# --- Univariate Analysis ---
numerical_cols = ['TotalClaims', 'TotalPremium', 'LossRatio', 'CustomValueEstimate']
for col in numerical_cols:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[col], bins=50, kde=True)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.tight_layout()
    plt.show()

NameError: name 'df' is not defined

<Figure size 800x400 with 0 Axes>

In [7]:
# --- Categorical Analysis ---
categorical_cols = ['Province', 'VehicleType', 'Gender']
for col in categorical_cols:
    plt.figure(figsize=(8, 4))
    sns.countplot(data=df, x=col, order=df[col].value_counts().index)
    plt.title(f"Count Plot of {col}")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

NameError: name 'df' is not defined

<Figure size 800x400 with 0 Axes>

In [8]:
# --- Bivariate Analysis ---
# Loss Ratio by Province
plt.figure(figsize=(10, 5))
df.groupby('Province')['LossRatio'].mean().sort_values().plot(kind='bar')
plt.title('Average Loss Ratio by Province')
plt.ylabel('Loss Ratio')
plt.tight_layout()
plt.show()


NameError: name 'df' is not defined

<Figure size 1000x500 with 0 Axes>

In [9]:
# Total Claims vs Total Premium
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='TotalPremium', y='TotalClaims', hue='Province')
plt.title('Total Claims vs Total Premium by Province')
plt.tight_layout()
plt.show()

NameError: name 'df' is not defined

<Figure size 800x600 with 0 Axes>

In [10]:
# Correlation heatmap
plt.figure(figsize=(10, 6))
corr_matrix = df[['TotalClaims', 'TotalPremium', 'CustomValueEstimate', 'LossRatio']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

NameError: name 'df' is not defined

<Figure size 1000x600 with 0 Axes>

In [11]:
# --- Outlier Detection ---
plt.figure(figsize=(10, 5))
sns.boxplot(data=df[numerical_cols])
plt.title('Boxplot of Key Numerical Features')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

NameError: name 'df' is not defined

<Figure size 1000x500 with 0 Axes>

In [ ]:
 Save a processed summary
summary = df[['Province', 'VehicleType', 'Gender', 'TotalClaims', 'TotalPremium', 'LossRatio']]
summary.to_csv("../data/processed/summary_stats.csv", index=False)

print("EDA complete. Summary file saved to: data/processed/summary_stats.csv")
